In [48]:
import pyodbc

# Kết nối tới master để tạo DB mới
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=AUSTINNGUYEN;'  # ae thay đổi tên máy chủ
    'DATABASE=master;'
    'Trusted_Connection=yes;',
    autocommit=True  #Quan trọng để tránh lỗi CREATE DATABASE
)

cursor = conn.cursor()
cursor.execute("IF DB_ID('StudentDB') IS NULL CREATE DATABASE StudentDB")
print("Database 'StudentDB' đã được tạo.")
# cursor.close()
# conn.close()


Database 'StudentDB' đã được tạo.


In [33]:
import pyodbc

# Connection: kết nối tới SQL Server
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=AUSTINNGUYEN;'
    'DATABASE=StudentDB;'
    'Trusted_Connection=yes;',
    autocommit=False  # Tắt autocommit để thử rollback/commit
)

# cursor: tạo đối tượng để thực hiện truy vấn
cursor = conn.cursor()

try:
    #execute(): thực thi lệnh SQL
    cursor.execute("""
        IF OBJECT_ID('students_test', 'U') IS NOT NULL DROP TABLE students_test;
        CREATE TABLE students_test (
            id INT PRIMARY KEY,
            name NVARCHAR(100),
            score INT
        )
    """)
    cursor.execute("INSERT INTO students_test (id, name, score) VALUES (?, ?, ?)", (1, 'Nguyen Van A', 90))
    cursor.execute("INSERT INTO students_test (id, name, score) VALUES (?, ?, ?)", (2, 'Tran Thi B', 95))

    # fetchall(): lấy dữ liệu sau SELECT
    cursor.execute("SELECT * FROM students_test")
    rows = cursor.fetchall()
    for row in rows:
        print(row)

    # commit(): xác nhận thay đổi vào database nếu không có lỗi
    conn.commit()
    print("Dữ liệu đã được lưu vào cơ sở dữ liệu.")

except Exception as e:
    print("Lỗi xảy ra:", e)

    # rollback(): huỷ bỏ thay đổi nếu có lỗi
    conn.rollback()
    print("Đã rollback.")

# finally:
#     # close(): đóng cursor và connection
#     cursor.close()
#     conn.close()
#     print("Kết thúc kết nối.")

(1, 'Nguyen Van A', 90)
(2, 'Tran Thi B', 95)
Dữ liệu đã được lưu vào cơ sở dữ liệu.


In [49]:
from sqlalchemy import create_engine
import urllib

params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=AUSTINNGUYEN;"
    "DATABASE=StudentDB;"
    "Trusted_Connection=yes;"
)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")


In [50]:
import pandas as pd

df = pd.read_csv("3. StudentsPerformance.csv", encoding="utf-8")

# Ghi dữ liệu vào SQL Server (tự tạo bảng nếu chưa có)
df.to_sql("students_performance", engine, if_exists="replace", index=False)
print("Đã ghi dữ liệu vào bảng 'students_performance' trong database 'StudentDB'.")
df


Đã ghi dữ liệu vào bảng 'students_performance' trong database 'StudentDB'.


,id,gender,race,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,1,female,group B,bachelor's degree,standard,none,72,72,74
1,2,female,group C,some college,standard,completed,69,90,88
2,3,female,group B,master's degree,standard,none,90,95,93
3,4,male,group A,associate's degree,free/reduced,none,47,57,44
4,5,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...,...
995,996,female,group E,master's degree,standard,completed,88,99,95
996,997,male,group C,high school,free/reduced,none,62,55,55
997,998,female,group C,high school,free/reduced,completed,59,71,65
998,999,female,group D,some college,standard,completed,68,78,77


In [5]:
#Ví dụ 1: Tính điểm trung bình của các môn học theo giới tính
df_check = pd.read_sql("SELECT gender, AVG(\"math score\") AS avg_math, AVG(\"reading score\") AS avg_reading, AVG(\"writing score\") AS avg_writing FROM students_performance GROUP BY gender", engine)
print("Dữ liệu trong bảng:")
print(df_check)

Dữ liệu trong bảng:
   gender  avg_math  avg_reading  avg_writing
0  female        63           72           72
1    male        68           65           63


In [6]:
#Ví dụ 2: Đếm số học sinh theo nhóm chủng tộc và trình độ học vấn của phụ huynh
df_check = pd.read_sql("SELECT race, \"parental level of education\", COUNT(*) AS count FROM students_performance GROUP BY race, \"parental level of education\"", engine)
print("Dữ liệu trong bảng:")
print(df_check)

Dữ liệu trong bảng:
       race parental level of education  count
0   group A          associate's degree     14
1   group B          associate's degree     41
2   group C          associate's degree     78
3   group D          associate's degree     50
4   group E          associate's degree     39
5   group A           bachelor's degree     12
6   group B           bachelor's degree     20
7   group C           bachelor's degree     40
8   group D           bachelor's degree     28
9   group E           bachelor's degree     18
10  group A                 high school     18
11  group B                 high school     48
12  group C                 high school     64
13  group D                 high school     44
14  group E                 high school     22
15  group A             master's degree      3
16  group B             master's degree      6
17  group C             master's degree     19
18  group D             master's degree     23
19  group E             master's degree 

In [7]:
#Ví dụ 3: Tìm học sinh có điểm toán cao nhất
df_check = pd.read_sql("SELECT * FROM students_performance WHERE \"math score\" = (SELECT MAX(\"math score\") FROM students_performance)", engine)
print("Dữ liệu trong bảng:")
print(df_check)

Dữ liệu trong bảng:
    id  gender     race parental level of education         lunch  \
0  150    male  group E          associate's degree  free/reduced   
1  452  female  group E                some college      standard   
2  459  female  group E           bachelor's degree      standard   
3  624    male  group A                some college      standard   
4  626    male  group D                some college      standard   
5  917    male  group E           bachelor's degree      standard   
6  963  female  group E          associate's degree      standard   

  test preparation course  math score  reading score  writing score  
0               completed         100            100             93  
1                    none         100             92             97  
2                    none         100            100            100  
3               completed         100             96             86  
4               completed         100             97             99  
5      

In [8]:
#Ví dụ 4: Lọc học sinh có điểm đọc và viết trên 80
df_check = pd.read_sql("SELECT * FROM students_performance WHERE \"reading score\" > 80 AND \"writing score\" > 80", engine)
print("Dữ liệu trong bảng:")
print(df_check)

Dữ liệu trong bảng:
       id  gender     race parental level of education         lunch  \
0       2  female  group C                some college      standard   
1       3  female  group B             master's degree      standard   
2       7  female  group B                some college      standard   
3      17    male  group C                 high school      standard   
4      35    male  group E                some college      standard   
..    ...     ...      ...                         ...           ...   
180   983    male  group B            some high school      standard   
181   984  female  group A                some college      standard   
182   990  female  group D                some college  free/reduced   
183   996  female  group E             master's degree      standard   
184  1000  female  group D                some college  free/reduced   

    test preparation course  math score  reading score  writing score  
0                 completed          69    

In [9]:
#Ví dụ 5: Điểm trung bình của học sinh hoàn thành khóa học chuẩn bị kiểm tra
df_check = pd.read_sql("SELECT AVG(\"math score\") AS avg_math, AVG(\"reading score\") AS avg_reading, AVG(\"writing score\") AS avg_writing FROM students_performance WHERE \"test preparation course\" = 'completed'", engine)
print("Dữ liệu trong bảng:")
print(df_check)

Dữ liệu trong bảng:
   avg_math  avg_reading  avg_writing
0        69           73           74


In [10]:
%load_ext sql
%sql mssql+pyodbc:///?odbc_connect={params}


'Connected: @'

In [11]:
%%sql
INSERT INTO students_performance (
    gender, race, "parental level of education",
    lunch, "test preparation course", "math score",
    "reading score", "writing score"
) VALUES (
    'female', 'group C', 'bachelor''s degree',
    'standard', 'completed', 85, 90, 88
);

 * mssql+pyodbc:///?odbc_connect=DRIVER%3D%7BODBC+Driver+17+for+SQL+Server%7D%3BSERVER%3DAUSTINNGUYEN%3BDATABASE%3DStudentDB%3BTrusted_Connection%3Dyes%3B
1 rows affected.


[]

In [12]:
%%sql

SELECT TOP 10 * FROM students_performance


 * mssql+pyodbc:///?odbc_connect=DRIVER%3D%7BODBC+Driver+17+for+SQL+Server%7D%3BSERVER%3DAUSTINNGUYEN%3BDATABASE%3DStudentDB%3BTrusted_Connection%3Dyes%3B
Done.


id,gender,race,parental level of education,lunch,test preparation course,math score,reading score,writing score
1,female,group B,bachelor's degree,standard,none,72,72,74
2,female,group C,some college,standard,completed,69,90,88
3,female,group B,master's degree,standard,none,90,95,93
4,male,group A,associate's degree,free/reduced,none,47,57,44
5,male,group C,some college,standard,none,76,78,75
6,female,group B,associate's degree,standard,none,71,83,78
7,female,group B,some college,standard,completed,88,95,92
8,male,group B,some college,free/reduced,none,40,43,39
9,male,group D,high school,free/reduced,completed,64,64,67
10,female,group B,high school,free/reduced,none,38,60,50


In [73]:
import pyodbc

# Kết nối tới database StudentDB
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=AUSTINNGUYEN;'
    'DATABASE=StudentDB;'
    'Trusted_Connection=yes;',
    autocommit=True
)

cursor = conn.cursor()


In [ ]:
df

In [65]:
cursor.execute("""
IF OBJECT_ID('performance_log', 'U') IS NOT NULL DROP TABLE performance_log;
CREATE TABLE performance_log (
    log_id INT IDENTITY PRIMARY KEY,
    student_id INT,
    log_time DATETIME DEFAULT GETDATE()
)
""")
print("Đã tạo bảng performance_log.")


Đã tạo bảng performance_log.


In [66]:
# 1. Xoá trigger nếu tồn tại 
cursor.execute("""
IF OBJECT_ID('trg_log_insert', 'TR') IS NOT NULL
DROP TRIGGER trg_log_insert;
""")

# 2. Tạo trigger mới 
cursor.execute("""
CREATE TRIGGER trg_log_insert
ON students_performance
AFTER INSERT
AS
BEGIN
    INSERT INTO performance_log (student_id)
    SELECT id FROM inserted;
END
""")
 
print("Trigger đã được tạo thành công.")



Trigger đã được tạo thành công.


In [67]:
cursor.execute("""
INSERT INTO students_performance 
(id, gender, race, [parental level of education], lunch, [test preparation course], [math score], [reading score], [writing score])
VALUES (1001, 'male', 'group D', 'bachelor''s degree', 'standard', 'completed', 33, 89, 28)
""")

In [70]:
cursor.execute("SELECT * FROM performance_log")
rows = cursor.fetchall()

print("Nội dung bảng performance_log:")
for row in rows:
    print(row)


Nội dung bảng performance_log:
(1, 1001, datetime.datetime(2025, 6, 18, 9, 26, 58, 810000))


In [ ]:
cursor.close()
conn.close()
# Kết thúc kết nối

In [ ]:
df['average_score'] = df[['math score', 'reading_score', 'writing_score']].mean(axis=1)

In [ ]:
cursor.execute("""
ALTER TABLE students_performance
ADD average_score FLOAT
""")
    

In [ ]:
cursor.execute("""
UPDATE students_performance
SET average_score = ([math score] + [reading score] + [writing score]) / 3
""")

ProgrammingError: ('42000', '[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The AVG function requires 1 argument(s). (174) (SQLExecDirectW)')

In [76]:
df = pd.read_sql("SELECT * FROM students_performance", conn)
df

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_30884\224185088.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM students_performance", conn)


,id,gender,race,parental level of education,lunch,test preparation course,math score,reading score,writing score,average_score
0,1,female,group B,bachelor's degree,standard,none,72,72,74,72.0
1,2,female,group C,some college,standard,completed,69,90,88,82.0
2,3,female,group B,master's degree,standard,none,90,95,93,92.0
3,4,male,group A,associate's degree,free/reduced,none,47,57,44,49.0
4,5,male,group C,some college,standard,none,76,78,75,76.0
...,...,...,...,...,...,...,...,...,...,...
997,998,female,group C,high school,free/reduced,completed,59,71,65,65.0
998,999,female,group D,some college,standard,completed,68,78,77,74.0
999,1000,female,group D,some college,free/reduced,none,77,86,86,83.0
1000,1001,male,group D,bachelor's degree,standard,completed,33,89,28,50.0
